# KPI Automation

Click "Run All" and type in the needed information in the desired format.
The final KPI calculations are shown at the very end, after the last cell.

## Export CSV from Qualtrics Stakeholder survey

The following cell is to Auto-Reload Data files

In [ ]:
%reload_ext autoreload
%autoreload 2

### Add the CSV to your workspace, name it what you would like, and type it into the file_name variable as a string

Turn CSV data into a dataframe, make a dataframe with only needed columns

In [ ]:
import pandas as pd


file_name: str = "october_data.csv"

all_data = pd.read_csv(file_name)

all_data.head()

Make a new dataframe from the data, this time only taking the needed columns and rows (the first two rows were unneeded)

In [ ]:
all_data.drop([0,1], axis = 0, inplace = True)

data = all_data.reset_index()[["StartDate", "EndDate", "Q1", "Q2", "Q3", "Q4", "PMO - Led", "Quarter"]].copy()

data.head()

Create a new column in the dataframe for the year of the response based on the EndDate field

In [ ]:
year = list()
i: int = 0
while i < len(data):
    year.append(data["EndDate"][i][0:4])
    i += 1

data["Year"] = year

data.head()

Create a new column in the dataframe for the month of the response based on the EndDate field

In [ ]:
month = list()
i: int = 0
while i < len(data):
    month.append(data["EndDate"][i][5:7])
    i += 1

data["Month"] = month

data.head()

Making use of the month column, we make a Quarter column (denoted as Q so as to not be confused with the Qualtrics data field)

In [ ]:
quarter = list()
i: int = 0
while i < len(data):
    if data["Month"][i] == "07" or data["Month"][i] == "08" or data["Month"][i] == "09":
        quarter.append("1")
    elif data["Month"][i] == "10" or data["Month"][i] == "11" or data["Month"][i] == "12":
        quarter.append("2")
    elif data["Month"][i] == "01" or data["Month"][i] == "02" or data["Month"][i] == "03":
        quarter.append("3")
    elif data["Month"][i] == "04" or data["Month"][i] == "05" or data["Month"][i] == "06":
        quarter.append("4")
    i += 1

data["Q"] = quarter

data.head()

Calculate Fiscal Year based off of the month and the year

In [ ]:
fiscal_year: list[int] = list()

i: int = 0
while i < len(data):
    fiscal_year.append(data["Year"][i][2:4])
    fiscal_year[i] = int(fiscal_year[i])
    i += 1

# if data["Month"] == "01" or data["Month"] == "02" or data["Month"] == "03" or data["Month"] == "04" or data["Month"] == "05" or data["Month"] == "06":
#     fiscal_year = data["Year"]
i: int = 0
while i < len(data):
    if data["Month"][i] == "07" or data["Month"][i] == "08" or data["Month"][i] == "09" or data["Month"][i] == "10" or data["Month"][i] == "11" or data["Month"][i] == "12":
        fiscal_year[i] += 1
    i += 1

fiscal_year_str: list[str] = list()
for item in fiscal_year:
    fiscal_year_str.append(str(item))


data["Fiscal Year"] = fiscal_year_str

data.head()

Filter based off of Fiscal Year

In [ ]:
needed_fy: str = input("What is the needed Fiscal year? (Enter the last two digits of the Fiscal Year)")

fy_filter: list[bool] = data["Fiscal Year"] == needed_fy

data_in_year = data[fy_filter]

data_in_year.head()

Filter based on Quarter

In [ ]:
needed_quarter: str = input("What is your target quarter? (Type a number 1-4) If you would like to select the whole year, do not give a response.")

if needed_quarter != "":
    quarter_filter = data_in_year["Q"] == needed_quarter
    data_in_quarter = data_in_year[quarter_filter]
else: 
    data_in_quarter = data_in_year

data_in_quarter.head()

Filter based on PMO

In [ ]:
pmo_led = data["PMO - Led"] == "Yes"
non_pmo_led = data["PMO - Led"] == "No"
pmo_led_string: str = input("PMO-Led? (Enter 'Y' (yes) 'N' (no) or 'B' (both)): ")
pmo_filter: list[bool]

while pmo_led_string != "Y" and pmo_led_string != "N" and pmo_led_string != "B":
    pmo_led_string: str = input("PMO-Led? (Please enter 'Y' or 'N'): ")
if pmo_led_string == "Y":
    pmo_filter = data_in_quarter["PMO - Led"] == "Yes"
    unindexed_pmo = data_in_quarter[pmo_filter]
    filtered_by_pmo = unindexed_pmo.reset_index()
elif pmo_led_string == "N":
    pmo_filter = data_in_quarter["PMO - Led"] == "No"
    unindexed_pmo = data_in_quarter[pmo_filter]
    filtered_by_pmo = unindexed_pmo.reset_index()
elif pmo_led_string == "B":
    filtered_by_pmo = data_in_quarter.reset_index()




filtered_by_pmo

Assign a score for each question based on Likert Scale

In [ ]:
# filtered_by_pmo["Q1"][0]

In [ ]:

def score_assign(question: str):

    score: list[int] = list()
    i: int = 0
    while i < len(filtered_by_pmo):
        if filtered_by_pmo[question][i] == "Strongly agree":
            score.append(5)
        elif filtered_by_pmo[question][i] == "Somewhat agree":
            score.append(4)
        elif filtered_by_pmo[question][i] == "Neither agree nor disagree":
            score.append(3)
        elif filtered_by_pmo[question][i] == "Somewhat disagree":
            score.append(2)
        elif filtered_by_pmo[question][i] == "Strongly disagree":
            score.append(1)
        i += 1

    score_str = list()
    for item in score:
        score_str.append(item)
    return score_str


filtered_by_pmo["Q1 Score"] = score_assign("Q1")
filtered_by_pmo["Q2 Score"] = score_assign("Q2")
filtered_by_pmo["Q3 Score"] = score_assign("Q3")

filtered_by_pmo


Calculate KPIs!

First, calculate the sum of each column of scores

In [ ]:
def score_sum(score: str) -> float:
    """calculates the sum of each column of scores"""
    sum: float = 0
    k: int = 0
    while k < len(filtered_by_pmo):
        sum += filtered_by_pmo[score][k]
        k += 1
    return sum

Client Satisfaction

In [ ]:
client_satisfaction: str = str((score_sum("Q1 Score") + score_sum("Q2 Score") + score_sum("Q3 Score"))  / (len(filtered_by_pmo) * 3))


PM Communicaitons

In [ ]:
pm_communications: str = str(score_sum("Q2 Score")/ (len(filtered_by_pmo)))

Change Mgmt.

In [ ]:
change_mgmt: str = str(score_sum("Q3 Score") / (len(filtered_by_pmo)))

All metrics printed:

In [ ]:
print("Client satisfaction: " + client_satisfaction + " PM Communication: " + pm_communications + " Change mgmt. " + change_mgmt)